In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from sbi.inference import prepare_for_sbi, SNLE
from sbi.simulators.linear_gaussian import diagonal_linear_gaussian
import sbi.utils as sbi_utils

pyknos/nflows/nn/nde


In [6]:
import pickle

In [9]:
with open('file', 'rb') as f:
    x = pickle.load(f, encoding='latin1')

FileNotFoundError: [Errno 2] No such file or directory: 'file'

### Load prior min and max and gt

In [3]:
import sys
sys.path.append('../')

In [4]:
from parameter_setup import load_ground_truth_params, load_prior_min, load_prior_max

### Create dummy data

In [5]:
num_samples = 10000
x = torch.randn(num_samples, 13)
theta = torch.randn(num_samples, 35)

### Standardize data

In [6]:
x_mean = x.mean(dim=0)
x_std = x.std(dim=0)

theta_mean = theta.mean(dim=0)
theta_std = theta.std(dim=0)

In [7]:
theta_mean.shape

torch.Size([35])

In [8]:
x = (x-x_mean) / x_std
theta = (theta-theta_mean) / theta_std

In [9]:
x.shape

torch.Size([10000, 13])

### Run simulator to get x_o

In [10]:
x_o = torch.zeros(1, 13)

x_o = (x_o - x_mean) / x_std

### Create dummy simulator and dummy prior

In [11]:
def dummy_simulator(theta):
    return torch.ones(1,13)

dummy_prior = sbi_utils.BoxUniform(torch.as_tensor(load_prior_min()), torch.as_tensor(load_prior_max()))
_bound = torch.sqrt(torch.as_tensor(3.))
dummy_prior_norm = sbi_utils.BoxUniform(-_bound*torch.ones(35), _bound*torch.ones(35))

In [21]:
simulator, prior, x_shape = prepare_for_sbi(dummy_simulator, dummy_prior_norm)

inference = SNLE(
    simulator, 
    prior, 
    x_shape,  
    external_data=(theta, x), 
    mcmc_method='slice', 
    density_estimator='nsf'
)

In [ ]:
posterior = inference(
    num_rounds=1,
    num_simulations_per_round=0,
    max_num_epochs=5
)

### Analyse posterior

In [16]:
posterior = posterior.set_default_x(x=x_o)

In [17]:
posterior._mcmc_method = 'slice'

In [19]:
samples = posterior.sample((10000,), num_chains=20, show_progress_bars=True, thin=1)

# denormalize the samples
samples = samples * theta_std + theta_mean

# convert to list for pickling
samples_list = samples.numpy().tolist()

In [29]:
import pickle
with open('../results/posteriors/200616_dummyPosteriorSNL.pickle', 'wb') as handle:
    pickle.dump(posterior, handle)

In [36]:
with open('../results/samples/200616_dummyPosteriorSNL_samples.pickle', 'wb') as handle:
    pickle.dump(samples_list, handle, protocol=2)

### Posterior predictives
Has to be done in a different virtual env with python 2.